# Extração Automatizada de Dados de Biópsia Mamária - Pardini

## Introdução Técnica Detalhada

Este notebook implementa um pipeline completo de extração, processamento e persistência de dados de laudos de biópsia mamária provenientes da base de dados Pardini. O foco principal é identificar ocorrências de carcinoma em laudos relacionados à mama, extraindo informações estruturadas que podem ser utilizadas para análises clínicas e epidemiológicas.

### Objetivo Principal

O objetivo principal deste notebook é:
- Identificar laudos de biópsia mamária que contenham menção a carcinoma
- Extrair e estruturar essas informações de forma automatizada
- Persistir os resultados em uma tabela Delta para análises posteriores
- Monitorar o processo e gerar alertas em caso de falhas

Este processo é parte de uma solução mais ampla de saúde preventiva, focada especificamente na detecção precoce de câncer de mama através da análise automatizada de laudos médicos.

### Tecnologias Utilizadas

O notebook utiliza diversas tecnologias modernas:

- **Apache Spark / PySpark**: Framework de processamento distribuído para manipulação de grandes volumes de dados
  - `pyspark.sql`: Para manipulação de dados estruturados (DataFrames)
  - `pyspark.sql.functions`: Funções para transformação de dados
  - `pyspark.sql.types`: Definição de tipos de dados

- **Delta Lake**: Tecnologia de armazenamento para tabelas transacionais no data lake
  - `delta.tables.DeltaTable`: Para operações de merge/upsert em tabelas Delta

- **OctoOps**: Framework interno para orquestração e monitoramento de pipelines de dados
  - `Sentinel`: Para envio de alertas e notificações
  - `FeatureStoreManager`: Para gerenciamento de features

- **Bibliotecas Python Padrão**:
  - `logging`: Para registro de eventos
  - `datetime`: Para manipulação de datas
  - `traceback`: Para captura e formatação de erros

### Fluxo de Trabalho/Etapas Principais

O notebook segue um fluxo de trabalho sequencial e lógico:

1. **Configuração do Ambiente**:
   - Instalação de dependências (octoops)
   - Importação de bibliotecas necessárias
   - Configuração de logging

2. **Definição de Filtros e Parâmetros**:
   - Especificação da tabela de destino
   - Definição de cláusulas WHERE para filtrar dados incrementais
   - Configuração de critérios de extração (linha de cuidado, siglas de exame, faixa etária)

3. **Extração e Transformação de Dados**:
   - Construção de consulta SQL com expressões regulares para identificação de carcinoma
   - Extração dos dados da tabela fonte `refined.saude_preventiva.pardini_laudos`
   - Cálculo de idade e aplicação de transformações

4. **Validação dos Dados Extraídos**:
   - Contagem de registros
   - Verificação das siglas de exame disponíveis

5. **Persistência dos Dados Processados**:
   - Seleção das colunas relevantes
   - Implementação de operação merge/upsert na tabela Delta
   - Tratamento de erros e notificação

### Dados Envolvidos

#### Fonte de Dados
- **Tabela Fonte**: `refined.saude_preventiva.pardini_laudos`
  - Contém laudos médicos de exames realizados no laboratório Pardini
  - Inclui informações sobre pacientes, exames e resultados em formato semi-estruturado

#### Colunas Importantes
- `id_marca`, `id_unidade`, `id_cliente`: Identificadores do laboratório, unidade e paciente
- `id_ficha`, `ficha`, `id_item`, `id_subitem`: Identificadores do exame
- `sigla_exame`: Código que identifica o tipo de exame realizado
- `dth_pedido`, `dth_resultado`: Datas de solicitação e resultado do exame
- `laudo_tratado`: Texto completo do laudo médico (conteúdo principal para análise)
- `linha_cuidado`: Categoria do exame (filtrado para 'mama')
- `sexo_cliente`, `idade_cliente`: Dados demográficos do paciente

#### Tabela de Destino
- **Tabela Delta**: `refined.saude_preventiva.pardini_laudos_mama_biopsia`
  - Armazena os resultados da extração com campos adicionais
  - Inclui flags como `HAS_CARCINOMA` e o texto extraído `RAW_CARCINOMA`

### Resultados/Saídas Esperadas

O notebook produz como resultado principal:

1. **DataFrame Processado** (`df_spk_biopsia`):
   - Contém todos os laudos de biópsia de mama filtrados
   - Inclui colunas derivadas como `RAW_CARCINOMA` e `HAS_CARCINOMA`

2. **Tabela Delta Persistida**:
   - Dados armazenados em formato Delta no caminho `refined.saude_preventiva.pardini_laudos_mama_biopsia`
   - Disponíveis para consulta SQL e análises posteriores

3. **Logs e Alertas**:
   - Mensagens de log durante a execução
   - Alertas via Sentinel em caso de falhas ou ausência de dados para processamento

### Pré-requisitos

Para executar este notebook com sucesso, são necessários:

- **Ambiente Databricks** configurado com:
  - Runtime que suporte PySpark e Delta Lake
  - Acesso às tabelas mencionadas no data lake
  - Permissões para leitura/escrita nas tabelas relevantes

- **Bibliotecas Instaladas**:
  - `octoops`: Para integração com sistema de monitoramento
  - `databricks-feature-store`: Para gerenciamento de feature store

- **Variáveis de Ambiente**:
  - Configurações necessárias para acesso ao ambiente Databricks

### Considerações Importantes/Observações

- **Processamento Incremental**: O notebook implementa uma estratégia de processamento incremental, processando apenas registros mais recentes que os já armazenados na tabela de destino.

- **Expressões Regulares**: A detecção de carcinoma utiliza expressões regulares complexas que procuram pelo termo "CARCINOMA" em contextos relacionados à "MAMA". A precisão desta detecção depende da qualidade e estrutura dos laudos.

- **Filtro Demográfico**: Aplica-se um filtro para selecionar apenas pacientes do sexo feminino (UPPER(sexo_cliente) = 'F') na faixa etária entre 40 e 75 anos.

- **Tipos de Exame**: O notebook filtra por siglas específicas de exames relacionados a biópsias mamárias (USPCOB, CBGEST, USPAFI, etc.), que podem variar dependendo do protocolo do laboratório.

- **Tratamento de Erros**: Implementa mecanismos robustos para captura e notificação de erros, garantindo que falhas sejam prontamente identificadas.

- **Feature Store**: O notebook inclui código comentado para integração com Databricks Feature Store, que pode ser habilitado conforme necessário.
  
-----------  

## Instalação de Dependências

Esta célula realiza a instalação das bibliotecas necessárias para a execução do notebook. Duas bibliotecas são mencionadas:

1. **databricks-feature-store**: Esta linha está comentada (`# %pip install databricks-feature-store -q`), indicando que a biblioteca está disponível no ambiente ou será instalada posteriormente conforme necessidade. Esta biblioteca permite interagir com o Databricks Feature Store, um repositório centralizado para armazenar e gerenciar features.

2. **octoops**: Esta biblioteca está sendo efetivamente instalada (`%pip install octoops`) e é uma ferramenta interna utilizada para operacionalização de modelos e pipelines, incluindo funcionalidades de monitoramento, alerta e gerenciamento de feature store.

A instalação usa o comando mágico `%pip` do Jupyter, que permite instalar pacotes Python diretamente no ambiente de execução do notebook. O flag `-q` (na linha comentada) indica instalação em modo "quiet", reduzindo a verbosidade da saída.

Esta célula precisa ser executada no início do notebook para garantir que todas as dependências estejam disponíveis para as células subsequentes.

In [ ]:
# %pip install databricks-feature-store -q
%pip install octoops

## Importação de Bibliotecas e Módulos

Esta célula realiza a importação de todas as bibliotecas e módulos necessários para o funcionamento do pipeline de extração e processamento de dados. As importações abrangem diversas categorias de funcionalidades:

### Manipulação de Dados com PySpark

- **Funções SQL e Manipulação de DataFrames**:
  - `pyspark.sql.functions`: Importação de funções como `col`, `year`, `month`, `dayofmonth`, `when`, `lit` e `expr` para manipulação de colunas e transformações de dados
  - `pyspark.sql.types`: Importação do tipo `DateType` para trabalhar com dados temporais
  - `pyspark.sql`: Importação da classe `DataFrame` para manipulação estruturada de dados
  - `datediff`, `to_date`, `to_timestamp`: Funções específicas para manipulação de datas

### Utilitários Python Padrão

- **Manipulação de Tempo e Datas**:
  - `datetime`: Módulo para manipulação de datas e horários
  
- **Logging e Tratamento de Erros**:
  - `logging`: Para registro de mensagens de log
  - `sys`: Acesso a variáveis e funcionalidades específicas do interpretador
  - `traceback`: Para captura e formatação de informações de exceções

### Ferramentas de Operacionalização (OctoOps)

- **Gerenciamento de Features e Monitoramento**:
  - `octoops.OctoOps`: Framework principal para operacionalização
  - `octoops.Sentinel`: Sistema de alerta e monitoramento
  - `octoops.FeatureStoreManager`: Gerenciador da feature store

- **Feature Store do Databricks**:
  - `databricks.feature_store.FeatureStoreClient`: Cliente para interação com o Feature Store do Databricks

Estas importações fornecem todas as ferramentas necessárias para implementar cada etapa do pipeline, desde a extração dos dados brutos até o processamento, transformação, persistência e monitoramento.

In [ ]:
from pyspark.sql.functions import col, year, month, dayofmonth, when, lit, expr, to_timestamp
from pyspark.sql.types import DateType
from pyspark.sql import DataFrame
from pyspark.sql.functions import datediff, to_date
from datetime import datetime
import logging
import sys
import traceback
from octoops import OctoOps
from octoops import Sentinel
from octoops import FeatureStoreManager
from databricks.feature_store import FeatureStoreClient

## Configuração do Logger

Esta célula configura um sistema de logging básico para o notebook, permitindo o registro de mensagens durante a execução. A configuração do logger é uma prática recomendada em desenvolvimento de pipelines de dados, pois facilita o diagnóstico de problemas e o monitoramento do fluxo de execução.

A linha de código utiliza o módulo `logging` do Python (importado anteriormente) para criar um logger específico para este módulo:

```python
logger = logging.getLogger(__name__)
```

O parâmetro `__name__` é uma variável especial do Python que contém o nome do módulo atual. Em um notebook Jupyter, isso geralmente corresponde a `__main__`, mas a prática de usar `__name__` é consistente com o desenvolvimento de módulos Python reutilizáveis.

Este logger será utilizado posteriormente para registrar informações, avisos e erros durante a execução do pipeline, permitindo:
- Rastreamento de eventos durante o processamento
- Identificação de problemas em etapas específicas
- Registro histórico da execução

Embora esta configuração seja simples, ela estabelece a estrutura básica para um sistema de logging que poderia ser expandido com configurações mais avançadas (como níveis de log, formatos personalizados ou redirecionamento para arquivos ou serviços externos).

In [ ]:
# O código configura um logger para registrar mensagens de log no módulo atual (__name__).
logger = logging.getLogger(__name__)

## Definição de Filtros para Extração de Dados

Esta célula configura os parâmetros e filtros que serão utilizados para extrair dados relevantes do banco de dados. Ela define três elementos principais:

### 1. Tabela de Destino

```python
table_biopsia = "refined.saude_preventiva.pardini_laudos_mama_biopsia"
```

Esta variável armazena o caminho completo da tabela Delta onde os dados processados serão persistidos. A tabela está localizada no schema `refined.saude_preventiva` e é específica para laudos de biópsia mamária do Pardini.

### 2. Cláusula WHERE para Processamento Incremental

```python
where_clause = f"""
    WHERE
        _datestamp > (
            SELECT MAX(biop._datestamp)
            FROM {table_biopsia} biop
        )
    """
```

Esta cláusula SQL implementa uma estratégia de processamento incremental, garantindo que apenas registros novos (com `_datestamp` mais recente que o último registro processado) sejam extraídos. Isso é fundamental para:
- Evitar o reprocessamento de dados já analisados
- Reduzir o custo computacional
- Manter a consistência dos dados

### 3. Filtro de Extração Detalhado

```python
filtro_extracao = """
    WHERE
        linha_cuidado   = 'mama'
        AND sigla_exame IN (...)
        AND UPPER(sexo_cliente) = 'F'
        AND (idade_cliente >= 40 AND idade_cliente < 76)
"""
```

Este filtro mais complexo implementa critérios específicos de seleção:

- **Linha de Cuidado**: Restringe a exames relacionados à linha de cuidado 'mama'
- **Siglas de Exame**: Seleciona apenas exames específicos relacionados a biópsia mamária:
  - "USPCOB": Ultrassonografia com core biopsy
  - "CBGEST": Core biopsy guiada por estereotaxia
  - "USPAFI": Ultrassonografia com punção aspirativa
  - "ESMATO": Estudo anatomopatológico mamário
  - "USMATO": Ultrassonografia mamária
  - "MPMFECG": Mamografia com nódulo palpável
  - "RMMATO": Ressonância magnética mamária
  - "USPMAF": Ultrassonografia de mama feminina
- **Demografia do Paciente**:
  - Apenas sexo feminino: `UPPER(sexo_cliente) = 'F'`
  - Faixa etária de 40 a 75 anos: `idade_cliente >= 40 AND idade_cliente < 76`

Estes filtros garantem que apenas dados relevantes para o objetivo do notebook (detecção de carcinoma em biópsias mamárias) sejam processados, otimizando o desempenho e a precisão da análise.

In [ ]:
# filtros para extração

table_biopsia = "refined.saude_preventiva.pardini_laudos_mama_biopsia"
 
where_clause = f"""
    WHERE
        _datestamp > (
            SELECT MAX(biop._datestamp)
            FROM {table_biopsia} biop
        )
    """

 
filtro_extracao = """
    WHERE
        linha_cuidado   = 'mama'
        AND sigla_exame IN (
            "USPCOB",
            "CBGEST",
            "USPAFI",
            "ESMATO",
            "USMATO",
            "MPMFECG",
            "RMMATO",
            "USPMAF"
        )
        AND UPPER(sexo_cliente) = 'F'
        AND (
            idade_cliente >= 40 AND idade_cliente < 76
        ) 
"""

## Consulta SQL para Extração e Transformação de Laudos

Esta célula contém a parte central do processamento de dados: uma consulta SQL complexa que extrai, filtra e transforma os dados de laudos, identificando a presença de carcinoma em laudos de biópsia mamária.

### Objetivo da Célula

Esta célula:
1. Define uma consulta SQL para extrair laudos relevantes da tabela fonte
2. Implementa expressões regulares para detecção de carcinoma
3. Cria colunas derivadas como `RAW_CARCINOMA` e `HAS_CARCINOMA`
4. Executa a consulta e armazena os resultados em um DataFrame
5. Exibe o DataFrame resultante para análise

### Estrutura da Consulta SQL

A consulta utiliza uma Common Table Expression (CTE) chamada `base` para organizar a lógica:

1. **Seleção de Colunas Originais**:
   - Identificadores: `id_marca`, `id_unidade`, `id_cliente`, `id_ficha`, `ficha`, etc.
   - Dados temporais: `dth_pedido`, `dth_resultado`, `_datestamp`
   - Conteúdo do laudo: `laudo_tratado`
   - Metadados: `linha_cuidado`, `sexo_cliente`

2. **Cálculo de Idade**:
   ```sql
   TIMESTAMPDIFF(DAY, flr.dth_nascimento_cliente, CURDATE()) / 365.25
   ```
   Calcula a idade em anos com precisão, usando a diferença em dias entre a data de nascimento e a data atual, dividida por 365.25 (considerando anos bissextos).

3. **Detecção de Carcinoma** usando expressões regulares:
   ```sql
   REGEXP_EXTRACT(
       flr.laudo_tratado,
       r'(?mi).*Topografia:.*MAMA.*(CARCINOMA).*|.*(CARCINOMA).*Topografia:.*MAMA.*',
       1
   ) AS RAW_CARCINOMA
   ```
   Esta expressão regular procura dois padrões:
   - "Topografia: MAMA" seguido por "CARCINOMA"
   - "CARCINOMA" seguido por "Topografia: MAMA"
   
   As flags `(?mi)` indicam modo multiline e case-insensitive.

4. **Flag de Presença de Carcinoma**:
   ```sql
   CASE
       WHEN REGEXP_EXTRACT(...) = ''
       THEN FALSE
       ELSE TRUE
   END AS HAS_CARCINOMA
   ```
   Cria uma coluna booleana que indica se carcinoma foi detectado no laudo.

### Aplicação dos Filtros

A consulta incorpora o filtro definido anteriormente (`filtro_extracao`), garantindo que apenas laudos relevantes sejam selecionados.

### Execução e Visualização

As últimas duas linhas:
1. Executam a consulta SQL usando o SparkSession e armazenam o resultado em `df_spk_biopsia`
2. Exibem o DataFrame resultante usando `display(df_spk_biopsia)`

### Impacto

Esta célula é fundamental para o pipeline, pois:
1. Implementa a lógica central de detecção de carcinoma
2. Cria os indicadores que serão utilizados para análises posteriores
3. Garante que apenas dados relevantes sejam processados, aplicando os filtros demográficos e de tipo de exame

In [ ]:
# Utiliza REGEXP_EXTRACT para buscar, no texto do campo 'laudo_tratado', 
# a ocorrência da palavra "CARCINOMA" relacionada à "MAMA" (seja antes ou depois de "Topografia").
# O padrão regex considera maiúsculas/minúsculas e múltiplas linhas.
# Se encontrar, extrai "CARCINOMA" para a coluna RAW_CARCINOMA; caso contrário, retorna string vazia.
# O CASE seguinte verifica se RAW_CARCINOMA está vazia:
# - Se estiver vazia, HAS_CARCINOMA recebe FALSE (não há carcinoma relacionado à mama no laudo).
# - Se não estiver vazia, HAS_CARCINOMA recebe TRUE (há carcinoma relacionado à mama no laudo).

query = f"""
WITH base AS (
    SELECT
        flr.id_marca,
        flr.id_unidade,
        flr.id_cliente, 
        flr.id_ficha,
        flr.ficha,
        flr.id_item, 
        flr.id_subitem, 
        flr.sigla_exame, 
        flr.dth_pedido,
        flr.dth_resultado,
        flr.sigla_exame,
        flr.laudo_tratado,
        flr.linha_cuidado,
        flr.sexo_cliente,
        flr.`_datestamp`,
        (
          TIMESTAMPDIFF(DAY, flr.dth_nascimento_cliente, CURDATE()) / 365.25
        ) AS idade_cliente,
        REGEXP_EXTRACT(
            flr.laudo_tratado,
            r'(?mi).*Topografia:.*MAMA.*(CARCINOMA).*|.*(CARCINOMA).*Topografia:.*MAMA.*',
            1
        ) AS RAW_CARCINOMA,
        CASE
            WHEN REGEXP_EXTRACT(
                    flr.laudo_tratado,
                    r'(?mi).*Topografia:.*MAMA.*(CARCINOMA).*|.*(CARCINOMA).*Topografia:.*MAMA.*',
                    1
                 ) = ''
            THEN FALSE
            ELSE TRUE
        END AS HAS_CARCINOMA
    FROM refined.saude_preventiva.pardini_laudos flr
    
)
SELECT *
FROM base
{filtro_extracao}
"""
df_spk_biopsia = spark.sql(query)
display(df_spk_biopsia)

## Contagem de Registros Extraídos

Esta célula executa uma operação simples mas importante: conta o número total de registros no DataFrame `df_spk` resultante da consulta SQL anterior. 

A função `count()` é uma operação de ação em Spark que percorre todos os registros no DataFrame e retorna o número total de linhas. Esta contagem é útil para:

1. **Validação da Extração**: Confirmar se a consulta SQL retornou algum resultado
2. **Monitoramento**: Acompanhar o volume de dados sendo processado
3. **Diagnóstico**: Identificar possíveis problemas na consulta ou nos filtros se o número for muito diferente do esperado

Note que há um problema potencial nesta célula: ela referencia um DataFrame `df_spk` que não foi explicitamente definido nas células anteriores (onde o DataFrame foi nomeado como `df_spk_biopsia`). Isso pode indicar uma inconsistência na nomenclatura ou que esta célula assume que `df_spk` já existe de alguma execução anterior ou outra parte do notebook.

Esta discrepância pode causar um erro do tipo `NameError: name 'df_spk' is not defined` se o DataFrame `df_spk` não existir no momento da execução.

In [ ]:
df_spk.count()

## Verificação de Siglas de Exame Disponíveis

Esta célula realiza uma análise exploratória simples mas importante: verifica quais siglas de exame estão efetivamente presentes nos dados extraídos. O comentário inicial explica o objetivo: "Não temos todas as siglas na base de dados", indicando que esta célula foi criada para verificar quais das siglas definidas no filtro estão realmente disponíveis nos dados.

### Objetivo da Célula

Esta verificação é importante porque:
1. Valida se os filtros estão funcionando corretamente
2. Identifica potenciais problemas de cobertura de dados
3. Fornece insights sobre a distribuição dos tipos de exame

### Operações Realizadas

A célula executa duas operações:

1. **Seleção e Distinção**:
   ```python
   df_spk.select("sigla_exame").distinct()
   ```
   Esta operação:
   - Seleciona apenas a coluna "sigla_exame" do DataFrame
   - Aplica a função `distinct()` para retornar apenas valores únicos (sem duplicatas)
   - O resultado é um novo DataFrame contendo apenas os valores únicos de siglas

2. **Visualização**:
   ```python
   display(df_spk.select("sigla_exame").distinct())
   ```
   Esta linha exibe o DataFrame resultante em formato tabular para análise visual.

### Considerações

Assim como na célula anterior, há uma inconsistência na nomenclatura do DataFrame - esta célula referencia `df_spk` enquanto o DataFrame criado anteriormente foi nomeado `df_spk_biopsia`. Esta discrepância pode causar um erro se `df_spk` não existir no momento da execução.

O resultado desta célula permite comparar quais das siglas definidas no filtro de extração (`"USPCOB"`, `"CBGEST"`, `"USPAFI"`, etc.) estão realmente presentes nos dados, identificando possíveis gaps na cobertura dos dados.

In [ ]:
# Não temos todas as siglas na base de dados
df_spk.select("sigla_exame").distinct()
display(df_spk.select("sigla_exame").distinct())

## Configuração do Feature Store (Comentado)

Esta célula contém código comentado para a configuração e criação de uma tabela no Databricks Feature Store. Embora o código não esteja ativo (está comentado), ele fornece a estrutura para persistir os dados processados como uma feature table gerenciada.

### Objetivo da Célula

O código, quando descomentado, criaria uma tabela no Feature Store do Databricks para armazenar os resultados da extração de forma gerenciada, com benefícios como:
- Rastreamento de linhagem de dados
- Versionamento
- Metadados e documentação
- Integração com MLflow e outras ferramentas do ecossistema Databricks

### Detalhes do Código Comentado

```python
# fs = FeatureStoreClient()
# fs.create_table(
#     name="refined.saude_preventiva.pardini_laudos_mama_biopsia",
#     primary_keys=["ficha",'id_item','id_subitem'],
#     schema=df_spk.schema,    
#     description="Jornada de Mama - Features extraídas de laudos de biopsia Pardini. Siglas: USPCOB, CBGEST, USPAFI, ESMATO, USMATO, MPMFECG, RMMATO, USPMAF")
```

Os parâmetros definidos incluem:

1. **Nome da Tabela**: `refined.saude_preventiva.pardini_laudos_mama_biopsia`
   - Define o local onde a tabela será criada no data lake

2. **Chaves Primárias**: `["ficha",'id_item','id_subitem']`
   - Define a combinação de colunas que identificam unicamente cada registro
   - Essencial para operações de merge/upsert posteriores

3. **Schema**: `df_spk.schema`
   - Utiliza o schema do DataFrame `df_spk` como modelo para a tabela
   - Isto garante compatibilidade de tipos de dados

4. **Descrição**: Documentação detalhada da tabela
   - Inclui o propósito (Features extraídas de laudos de biopsia Pardini)
   - Lista as siglas de exames relevantes

### Considerações

Como nas células anteriores, há uma inconsistência na nomenclatura do DataFrame - o código referencia `df_spk.schema` enquanto o DataFrame criado foi nomeado `df_spk_biopsia`.

Este código está comentado possivelmente porque:
- A tabela já foi criada em uma execução anterior
- Esta é uma versão de desenvolvimento/teste que ainda não deve persistir dados
- A estratégia de persistência foi alterada para o método Delta mostrado nas células posteriores

In [ ]:
# FeatureStore
# fs = FeatureStoreClient()
# fs.create_table(
#     name="refined.saude_preventiva.pardini_laudos_mama_biopsia",
#     primary_keys=["ficha",'id_item','id_subitem'],
#     schema=df_spk.schema,    
#     description="Jornada de Mama - Features extraídas de laudos de biopsia Pardini. Siglas: USPCOB, CBGEST, USPAFI, ESMATO, USMATO, MPMFECG, RMMATO, USPMAF")

## Processamento Final e Persistência dos Dados

Esta última célula é responsável pelo processamento final e salvamento dos dados extraídos na tabela Delta. É a célula mais complexa do notebook, implementando várias etapas críticas do pipeline:

### Objetivo da Célula

Esta célula:
1. Importa bibliotecas adicionais necessárias para tratamento de erros e manipulação de tabelas Delta
2. Define constantes para o sistema de monitoramento
3. Seleciona colunas específicas do DataFrame original
4. Implementa uma função para inserir/atualizar dados na tabela Delta
5. Executa o salvamento com tratamento de erros robusto
6. Envia alertas em caso de problemas

### Importações e Configurações Iniciais

```python
import sys
import traceback
from octoops import Sentinel
from delta.tables import DeltaTable

WEBHOOK_DS_AI_BUSINESS_STG = 'stg'

OUTPUT_DATA_PATH = 'refined.saude_preventiva.pardini_laudos_mama_biopsia'
```

Estas linhas:
- Importam módulos adicionais para tratamento de erros (`traceback`) e manipulação de tabelas Delta
- Definem o ambiente do webhook para o sistema de alerta (`stg` para staging/teste)
- Definem o caminho da tabela de saída (onde os dados serão persistidos)

### Seleção de Colunas

```python
df_spk = df_spk_biopsia.select("ficha","sequencial","sigla_exame","id_cliente","dth_pedido","dth_resultado", "laudo_tratado","linha_cuidado","_datestamp","RAW_CARCINOMA","HAS_CARCINOMA").display()
```

Esta linha:
- Seleciona apenas as colunas relevantes do DataFrame `df_spk_biopsia`
- Cria um novo DataFrame chamado `df_spk` com o subconjunto de colunas
- A função `.display()` ao final mostrará os dados na interface do notebook

### Função de Inserção de Dados

```python
def insert_data(df_spk, output_data_path):  
    # Carrega a tabela Delta existente
    delta_table = DeltaTable.forName(spark, output_data_path)

    # Faz o merge (upsert)
    (delta_table.alias("target")
     .merge(
         df_spk.alias("source"),
         "target.ficha = source.ficha AND target.sequencial = source.sequencial AND target.sigla_exame = source.sigla_exame"
     )
     .whenMatchedUpdateAll()
     .whenNotMatchedInsertAll()
     .execute())
```

Esta função implementa um padrão de merge (upsert) para tabelas Delta:
- Carrega a tabela Delta existente usando `DeltaTable.forName()`
- Configura uma operação de merge usando:
  - Aliases para as tabelas source e target
  - Condição de junção baseada nas chaves primárias
  - Regras para atualização (quando há correspondência) e inserção (quando não há)
- Executa a operação com `.execute()`

### Bloco Principal com Tratamento de Erros

O bloco `try/except` implementa o processamento com tratamento de erros:

1. **Verificação de Dados Disponíveis**:
   ```python
   if df_spk.count() > 0:
   ```
   Verifica se há registros para processar

2. **Salvamento de Dados** (quando há registros):
   ```python
   insert_data(df_spk, OUTPUT_DATA_PATH)
   print(f'Total de registros salvos na tabela: {df_spk.count()}')
   ```
   Chama a função de inserção e exibe uma mensagem de confirmação

3. **Notificação** (quando não há registros):
   ```python
   else:
       error_message = traceback.format_exc()
       summary_message = f"""Não há laudos para extração.\n{error_message}"""
       sentinela_ds_ai_business = Sentinel(...)
       sentinela_ds_ai_business.alerta_sentinela(...)
   ```
   Cria e envia um alerta usando o sistema Sentinel

4. **Tratamento de Exceções**:
   ```python
   except Exception as e:
       traceback.print_exc()        
       raise e
   ```
   Imprime o traceback completo e propaga a exceção para tratamento em nível superior

### Detalhes Adicionais

O código também inclui algumas linhas comentadas que indicam abordagens alternativas:
- Código comentado `#1/0` que parece ser um teste para forçar um erro
- Código comentado para salvar usando o Feature Store em vez da abordagem Delta direta

### Considerações

Esta célula representa o final do pipeline de processamento, garantindo que os dados sejam salvos de forma consistente e que problemas sejam prontamente notificados. A implementação do padrão de merge garante idempotência - o notebook pode ser executado múltiplas vezes sem criar duplicatas.

In [ ]:
import sys
import traceback
from octoops import Sentinel
from delta.tables import DeltaTable

WEBHOOK_DS_AI_BUSINESS_STG = 'stg'


#OUTPUT_DATA_PATH = dbutils.widgets.get('OUTPUT_DATA_PATH')
OUTPUT_DATA_PATH = 'refined.saude_preventiva.pardini_laudos_mama_biopsia'

# Selecionar colunas de interesse
df_spk = df_spk_biopsia.select("ficha","sequencial","sigla_exame","id_cliente","dth_pedido","dth_resultado", "laudo_tratado","linha_cuidado","_datestamp","RAW_CARCINOMA","HAS_CARCINOMA").display()

# função para salvar dados na tabela
def insert_data(df_spk, output_data_path):  

    # Carrega a tabela Delta existente
    delta_table = DeltaTable.forName(spark, output_data_path)

    # Faz o merge (upsert)
    (delta_table.alias("target")
     .merge(
         df_spk.alias("source"),
         "target.ficha = source.ficha AND target.sequencial = source.sequencial AND target.sigla_exame = source.sigla_exame"
     )
     .whenMatchedUpdateAll()
     .whenNotMatchedInsertAll()
     .execute())
            


# salvar dados
try:
    if df_spk.count() > 0:
        
        #1/0 
        insert_data(df_spk, OUTPUT_DATA_PATH)
        # fs.write_table(
    #     name="refined.saude_preventiva.fleury_laudos_mama_biopsia",
    #     df=df_spk,
    #     mode="merge",
        # )
        print(f'Total de registros salvos na tabela: {df_spk.count()}')
            
    else:
        error_message = traceback.format_exc()
        summary_message = f"""Não há laudos para extração.\n{error_message}"""
        sentinela_ds_ai_business = Sentinel(
            project_name='Monitor_Linhas_Cuidado_Torax',
            env_type=WEBHOOK_DS_AI_BUSINESS_STG,
            task_title='Pardini Mama Biopsia'
        )

        sentinela_ds_ai_business.alerta_sentinela(
            categoria='Alerta', 
            mensagem=summary_message,
            job_id_descritivo='2_pardini_mama_biopsia'
        )
except Exception as e:
    traceback.print_exc()        
    raise e